In [1]:
import os
import time
import motor.motor_asyncio
import poe_lib
import tqdm.notebook
import re
import numpy as np
import datetime

from poe_lib.objects import Note

In [2]:
client = motor.motor_asyncio.AsyncIOMotorClient(os.environ['MONGO_URL'])

In [3]:
def reduce_std(values, weights, n = 1, min_remaining = 10):
    an_array = values
    median = np.median(an_array)
    standard_deviation = np.std(an_array)
    distance_from_mean = abs(an_array - median)
    max_deviations = 2
    not_outlier = distance_from_mean < max_deviations * standard_deviation
    if np.count_nonzero(not_outlier) < min_remaining:
        return values, weights
    values = values[not_outlier]
    weights = weights[not_outlier]
    
    if n > 1:
        values, weights = reduce_std(values, weights, n - 1, min_remaining)
    
    return values, weights

In [4]:
async def gather_type(match_dict, mongo_name):
    
    print(f'Gather [{match_dict}] into [{mongo_name}]')
    
    
    
    pipeline = [
        {
            '$match': match_dict
        }, {
            '$project': {
                'typeLine': 1
            }
        }, {
            '$group': {
                '_id': None, 
                'types': {
                    '$addToSet': '$typeLine'
                }
            }
        }, {
            '$unwind': {
                'path': '$types'
            }
        }
    ]
    distinct_types = []
    async for distinct_type in client.trade.items.aggregate(pipeline):
        distinct_types.append(distinct_type['types'])

    await client.trade.meta.update_one(
        {'name': mongo_name},
        {'$set': {'name': mongo_name, 'types': sorted(distinct_types)}},
        upsert = True,
    )

In [5]:
t0 = time.time()
banned_items = [
    'Alteration Shard',
    'Scroll of Wisdom',
    'Transmutation Shard',
    'Alchemy Shard',
    'Ancient Shard',
    'Annulment Shard',
    'Binding Shard',
    'Chaos Shard',
    'Engineer\'s Shard',
    'Exalted Shard',
    'Harbinger\'s Shard',
    'Horizon Shard',
    'Regal Shard',
    'Transmutation Shard',
]

await gather_type({f'extended.category': 'currency','typeLine': {'$nin': banned_items}}, 'currency_types')
await gather_type({f'extended.category': 'cards','typeLine': {'$nin': banned_items}}, 'card_types')
await gather_type({f'extended.subcategories': 'fragment','typeLine': {'$nin': banned_items}}, 'fragment_types')
await gather_type({f'extended.subcategories': 'fossil','typeLine': {'$nin': banned_items}}, 'fossil_types')
print(time.time()-t0)

Gather [{'extended.category': 'currency', 'typeLine': {'$nin': ['Alteration Shard', 'Scroll of Wisdom', 'Transmutation Shard', 'Alchemy Shard', 'Ancient Shard', 'Annulment Shard', 'Binding Shard', 'Chaos Shard', "Engineer's Shard", 'Exalted Shard', "Harbinger's Shard", 'Horizon Shard', 'Regal Shard', 'Transmutation Shard']}}] into [currency_types]
Gather [{'extended.category': 'cards', 'typeLine': {'$nin': ['Alteration Shard', 'Scroll of Wisdom', 'Transmutation Shard', 'Alchemy Shard', 'Ancient Shard', 'Annulment Shard', 'Binding Shard', 'Chaos Shard', "Engineer's Shard", 'Exalted Shard', "Harbinger's Shard", 'Horizon Shard', 'Regal Shard', 'Transmutation Shard']}}] into [card_types]
Gather [{'extended.subcategories': 'fragment', 'typeLine': {'$nin': ['Alteration Shard', 'Scroll of Wisdom', 'Transmutation Shard', 'Alchemy Shard', 'Ancient Shard', 'Annulment Shard', 'Binding Shard', 'Chaos Shard', "Engineer's Shard", 'Exalted Shard', "Harbinger's Shard", 'Horizon Shard', 'Regal Shard', 

In [6]:
x = await client.trade.items.distinct('extended.subcategories')
x

[None,
 'abyss',
 'activegem',
 'amulet',
 'beast',
 'belt',
 'blueprint',
 'boots',
 'bow',
 'chest',
 'claw',
 'cluster',
 'contract',
 'dagger',
 'fossil',
 'fragment',
 'gloves',
 'heistobjective',
 'heistreward',
 'heisttool',
 'heistutility',
 'heistweapon',
 'helmet',
 'incubator',
 'invitation',
 'oneaxe',
 'onemace',
 'onesword',
 'piece',
 'quiver',
 'resonator',
 'ring',
 'rod',
 'runedagger',
 'sample',
 'scarab',
 'sceptre',
 'shield',
 'staff',
 'supportgem',
 'supportgemplus',
 'trinket',
 'twoaxe',
 'twomace',
 'twosword',
 'wand',
 'warstaff']

In [13]:
pricing_types = (await client.trade.meta.find_one({'name': 'currency_types'}))['types']
pricing_types += (await client.trade.meta.find_one({'name': 'card_types'}))['types']
pricing_types += (await client.trade.meta.find_one({'name': 'fragment_types'}))['types']
pricing_types += (await client.trade.meta.find_one({'name': 'fossil_types'}))['types']

# Build Currency Mapping

In [14]:
currency_map = {}
for currency in tqdm.notebook.tqdm((await client.trade.meta.find_one({'name': 'currency_types'}))['types'], unit='item'):
    cost_estimation = await client.trade.cost_estimates.find_one({'typeLine': currency})
    if cost_estimation is None:
        continue
    
    currency_map[currency] = cost_estimation['value']

  0%|          | 0/622 [00:00<?, ?item/s]

In [10]:
currency_map

{'A Call into the Void': 2.0833333333333335,
 'A Dishonourable Death': 15.0,
 'A Firm Foothold': 1.0,
 'A Forest of False Idols': 1.0,
 'A Master Seeks Help': 4.261798599142708,
 'A Prodigious Hand': 3.923076923076923,
 'A Regal Death': 1.0,
 'A Rift in Time': 2.2583699082982087,
 'A Valuable Combination': 1.0,
 'A Vision of Ice and Fire': 3.0,
 'A Whispered Prayer': 2.135135135135135,
 'Abberathine Horns': 5.732272727272727,
 'Aberrant Fossil': 1.999999999767807,
 'Abnormal Effulgence': 2.8666666666666667,
 'Abrasive Catalyst': 0.8041066909094079,
 'Abyssal Delirium Orb': 12.085891891891892,
 'Abyssal Incubator': 2.8728822453035274,
 'Accelerating Catalyst': 0.9999472511144132,
 "Admiral Proclar's Pipe": 6.251304347826087,
 'Aetheric Fossil': 2.0,
 'Against the Tide': 1.3436305493360994,
 'Agony at Dusk': 2.0,
 'Albino Rhoa Feather': 64.93956854123924,
 'Alchemical Chalice': 5.8733685772472874,
 'Amber Oil': 0.5152266111425805,
 'Amorphous Delirium Orb': 15.176923043478261,
 'An Unsee

In [15]:
for item_type in tqdm.notebook.tqdm(pricing_types, unit='item'):
    values = []
    my_filter = {'typeLine': item_type, '_price.value': {'$ne': None}, 'league': 'Scourge'}
    async for item in client.trade.items.find(my_filter, projection={'_price': 1, 'stackSize': 1}):
        
        if item['_price']['unit'] != 'Chaos Orb':
            if item['_price']['unit'] in currency_map:
                item['_price']['value'] *= currency_map[item['_price']['unit']]
        
        values.append((item['_price']['value'], item.get('stackSize', 1)))
    if len(values) == 0:
        continue
    count = len(values)
    values = sorted(values, key=lambda x: x[0])
    x = np.array(values)
    values = x.T[0]
    weights = x.T[1]
    values, weights = reduce_std(values, weights, n=10)
    await client.trade.cost_estimates.update_one(
        {'typeLine': item_type},
        {
            '$set': {
                'typeLine': item_type,
                'value': np.average(values, weights=weights),
                'std': np.std(values),
                'last_updated': datetime.datetime.utcnow(),
                'count': count,
            }
        },
        upsert=True,
    )

  0%|          | 0/1163 [00:00<?, ?item/s]